In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Import the files
data = pd.read_csv("Assistance_PrimeAwardSummaries.csv") 

grant = pd.DataFrame()

In [3]:
grant["grantId"] = data["award_id_fain"]

# grantName: Using prime_award_base_transaction_description
grant["grantName"] = data["prime_award_base_transaction_description"]

# funder: Agency for code, sub agency for name
grant["funderCode"] = data["awarding_agency_code"]
grant["funderAgency"] = data["awarding_sub_agency_name"]

# Amount: Using total_obligated_amount
grant["amount"] = data ["total_obligated_amount"]

# Declare dates
grant["startDate"] = data["period_of_performance_start_date"]
grant["endDate"] = data["period_of_performance_current_end_date"]

In [4]:
grant

,grantId,grantName,funderCode,funderAgency,amount,startDate,endDate
0,F32GM150204,MECHANISMS OF SEPTIN-ACTIN CYTOSKELETAL CROSST...,75,National Institutes of Health,71132.0,2023-09-29,NaN
1,F31AG079515,THE ROLE OF DOPAMINE IN COGNITIVE RESILIENCE T...,75,National Institutes of Health,33846.0,2023-11-03,NaN
2,2309635,COMPUTATIONAL MODELING TO DETERMINE STRATEGIES...,49,National Science Foundation,280000.0,2023-07-01,2026-06-30
3,P268K240184,2023-2024 DIRECT LOAN BASE RECORD,91,Department of Education,0.0,NaN,2045-07-31
4,R21DA058581,RAPID MEASUREMENT OF NOVEL HARM REDUCTION HOUS...,75,National Institutes of Health,263287.0,2023-05-01,2025-04-30
...,...,...,...,...,...,...,...
487,1120938,THE FUNCTIONAL ROLE OF EXPERIENCE IN THE DEVEL...,49,National Science Foundation,900000.0,2011-09-01,2016-08-31
488,1102935,GRADUATE RESEARCH FELLOWSHIP PROGRAM,49,National Science Foundation,384797.0,2010-12-01,2015-11-30
489,1068620,IGERT: GEOMETRY AND DYNAMICS -- INTEGRATED ED...,49,National Science Foundation,2867668.0,2011-08-15,2016-07-31
490,1053842,"CAREER: HOLOGRAPHY, QUANTUM INFORMATION, AND E...",49,National Science Foundation,448610.0,2011-08-01,2017-07-31


In [5]:
# Remove the Department of Education
DoE = grant[grant["funderAgency"]=="Department of Education"]

grant = grant[grant["funderAgency"]!="Department of Education"]

DoE.to_excel("Department of Education Grants.xlsx", index=False)

In [6]:
grant

,grantId,grantName,funderCode,funderAgency,amount,startDate,endDate
0,F32GM150204,MECHANISMS OF SEPTIN-ACTIN CYTOSKELETAL CROSST...,75,National Institutes of Health,71132.0,2023-09-29,NaN
1,F31AG079515,THE ROLE OF DOPAMINE IN COGNITIVE RESILIENCE T...,75,National Institutes of Health,33846.0,2023-11-03,NaN
2,2309635,COMPUTATIONAL MODELING TO DETERMINE STRATEGIES...,49,National Science Foundation,280000.0,2023-07-01,2026-06-30
4,R21DA058581,RAPID MEASUREMENT OF NOVEL HARM REDUCTION HOUS...,75,National Institutes of Health,263287.0,2023-05-01,2025-04-30
5,CH5074510,CONNECTING THE HUMANITIES: INNOVATIVE PROGRAMM...,418,National Endowment for the Humanities,573333.0,2007-12-01,2015-07-31
...,...,...,...,...,...,...,...
487,1120938,THE FUNCTIONAL ROLE OF EXPERIENCE IN THE DEVEL...,49,National Science Foundation,900000.0,2011-09-01,2016-08-31
488,1102935,GRADUATE RESEARCH FELLOWSHIP PROGRAM,49,National Science Foundation,384797.0,2010-12-01,2015-11-30
489,1068620,IGERT: GEOMETRY AND DYNAMICS -- INTEGRATED ED...,49,National Science Foundation,2867668.0,2011-08-15,2016-07-31
490,1053842,"CAREER: HOLOGRAPHY, QUANTUM INFORMATION, AND E...",49,National Science Foundation,448610.0,2011-08-01,2017-07-31


In [7]:
# Find how many grants belong to each agency
unique = grant["funderAgency"].unique()

for i in unique:
    print(i+",", len(grant[grant["funderAgency"]==i]))

National Institutes of Health, 217
National Science Foundation, 113
National Endowment for the Humanities, 4
National Aeronautics and Space Administration, 2
Department of the Air Force, 2
Department of Energy, 12
Administration for Community Living, 8
Federal Communications Commission, 3
National Endowment for the Arts, 1
Small Business Administration, 26
Assistant Secretary for Policy Development and Research, 2
Health Resources and Services Administration, 3
Administration for Children and Families, 5
United States Institute of Peace, 1
Administration on Aging, 4
Department of Defense, 6
Department of the Army, 1
Food and Drug Administration, 1
Agency for Healthcare Research and Quality, 2
Department of State, 1
Institute of Museum and Library Services, 2
Environmental Protection Agency, 1
Office of Justice Programs, 6


In [9]:
# Export Duplicates
duplicates = grant[grant["grantId"].duplicated(keep=False)].sort_values(by="grantId")

duplicates.to_excel("Duplicate Grants.xlsx", index=False)

Agency sites that work by calling in the grant number

NSF: https://www.nsf.gov/awardsearch/advancedSearch.jsp

National Endowment for the Humanities: https://securegrants.neh.gov/publicquery/  

National Aeronautics and Space Administration: https://www.nasa.gov/centers/nssc/forms/grant-status-form

In [11]:
# Find investigators for NIH API
import requests

def nih_api(code):
    url = "https://api.reporter.nih.gov/v2/projects/search"
    headers = {
        "accept": "application/json",
        "Content-Type": "application/json"
    }
    data = {
        "criteria": {
            "project_nums": [code+"*"]
        }
    }

    response = requests.post(url, headers=headers, json=data)
    return response.json()

In [12]:
# Find investigators for nsf API
import requests

def nsf_api(code):
    base_url = "http://api.nsf.gov/services/v1/awards/"
    award_id = code
    data_format = "json"

    url = base_url + f"{award_id}.{data_format}"
    headers = {
        "accept": "application/json"
    }

    response = requests.get(url, headers=headers)
    return response.json()

In [13]:
# Declare new columns
grant["keyword"] = ""
grant["description"] = ""
grant["investigatorId"] = ""

C:\Users\gmbab\AppData\Local\Temp\ipykernel_20540\438115459.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grant["keyword"] = ""
C:\Users\gmbab\AppData\Local\Temp\ipykernel_20540\438115459.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grant["description"] = ""
C:\Users\gmbab\AppData\Local\Temp\ipykernel_20540\438115459.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

In [14]:
grant = grant.reset_index()
grant = grant.drop(['index'], axis=1)

In [15]:
grant

,grantId,grantName,funderCode,funderAgency,amount,startDate,endDate,keyword,description,investigatorId
0,F32GM150204,MECHANISMS OF SEPTIN-ACTIN CYTOSKELETAL CROSST...,75,National Institutes of Health,71132.0,2023-09-29,NaN,,,
1,F31AG079515,THE ROLE OF DOPAMINE IN COGNITIVE RESILIENCE T...,75,National Institutes of Health,33846.0,2023-11-03,NaN,,,
2,2309635,COMPUTATIONAL MODELING TO DETERMINE STRATEGIES...,49,National Science Foundation,280000.0,2023-07-01,2026-06-30,,,
3,R21DA058581,RAPID MEASUREMENT OF NOVEL HARM REDUCTION HOUS...,75,National Institutes of Health,263287.0,2023-05-01,2025-04-30,,,
4,CH5074510,CONNECTING THE HUMANITIES: INNOVATIVE PROGRAMM...,418,National Endowment for the Humanities,573333.0,2007-12-01,2015-07-31,,,
...,...,...,...,...,...,...,...,...,...,...
418,1120938,THE FUNCTIONAL ROLE OF EXPERIENCE IN THE DEVEL...,49,National Science Foundation,900000.0,2011-09-01,2016-08-31,,,
419,1102935,GRADUATE RESEARCH FELLOWSHIP PROGRAM,49,National Science Foundation,384797.0,2010-12-01,2015-11-30,,,
420,1068620,IGERT: GEOMETRY AND DYNAMICS -- INTEGRATED ED...,49,National Science Foundation,2867668.0,2011-08-15,2016-07-31,,,
421,1053842,"CAREER: HOLOGRAPHY, QUANTUM INFORMATION, AND E...",49,National Science Foundation,448610.0,2011-08-01,2017-07-31,,,


In [27]:
# traverse data set and add information where appropriate to institution
for i in range(423):
    if grant.loc[i, "funderAgency"] == "National Institutes of Health":
        # Use API
        result  = nih_api(grant.loc[i, "grantId"])["results"][0]
        
        # Title
        grant.loc[i, "grantName"] = result['project_title']
        
        # Abstract
        grant.loc[i, "description"] = result["abstract_text"].removeprefix("PROJECT SUMMARY/ABSTRACT").removeprefix("PROJECT SUMMARY").removeprefix("ABSTRACT").removeprefix("\n")
        
        # Keywords
        grant.loc[i, "keyword"] = result["terms"].replace("<", "", 1).replace("<", " ").replace(">", ",")
                
        # PI
        grant.loc[i, "investigatorId"] = result["contact_pi_name"]
        
        print(i,  grant.loc[i, "investigatorId"])
        
    if grant.loc[i, "funderAgency"] == "National Science Foundation":
        result = nsf_api(grant.loc[i, "grantId"])['response']['award'][0]
        
        # Title
        grant.loc[i, "grantName"] = result["title"]
        
        # PI
        grant.loc[i, "investigatorId"] = result["piLastName"]+", "+result["piFirstName"]
        print(i,  grant.loc[i, "investigatorId"])
    print()

0 MAGLIOZZI, JOSEPH O

1 PARENT, JOURDAN 

2 Hagan, Michael

3 GREEN, TRACI C


5 KADENER, SEBASTIAN 

6 KRAUSS, ISAAC JONATHAN

7 KADENER, SEBASTIAN 

8 KRAUSS, ISAAC JONATHAN

9 Ragunathan, Kaushik

10 RODAL, AVITAL ADAH


12 MITRA, MONIKA 


14 ROSBASH, MICHAEL 

15 Chakraborty, Bulbul

16 DUCLOS YEVICK, HANNAH GABRIELLE

17 Van Hooser, Stephen

18 GARRITY, PAUL 




22 Baskaran, Aparna

23 OLSON, JACOB MICHAEL

24 Xue, Nianwen

25 Fai, Thomas

26 Rogers, William

27 BERRY, ANNE SHIVELY


29 Schmidt-Rohr, Klaus

30 GOODE, BRUCE L

31 MAIGLER, KATHLEEN C


33 Xu, Hao

34 PESCOSOLIDO, MATTHEW F.

35 Kleinbock, Dmitry

36 Bernardi, Olivier

37 REIF, SHARON 


39 GRIFFITH, LESLIE C

40 Han, Grace

41 Haber, James


43 STEWART, MAUREEN T

44 MITRA, MONIKA 



47 Wilson, Steven

48 Cadge, Wendy


50 Lovett, Susan

51 LAWSON, HANNAH N

52 Cutcher-Gershenfeld, Joel

53 Sengupta, Piali

54 Lovett, Susan

55 XU, BING 

56 Abbott, Carolyn

57 Nicholson, Joanne





62 Pustejovsky, James

63 Gi

In [33]:
# Export to Excell
grant.to_excel("Grants.xlsx", index=False)

In [29]:
# convert data to string and remove nulls
grant["endDate"] = grant["endDate"].astype(str)
grant.loc[grant["endDate"] == "nan", "endDate"] = ""

grant["startDate"] = grant["startDate"].astype(str)
grant.loc[grant["startDate"] == "nan", "startDate"] = ""

In [30]:
df = grant

In [31]:
# Export grants to xml

import xml.etree.ElementTree as ET


# Create the XML structure
root = ET.Element("grants")
for _, row in df.iterrows():
    grant = ET.SubElement(root, "grant")
    grantId = ET.SubElement(grant, "grantId")
    grantId.text = row['grantId']
    grantName = ET.SubElement(grant, "grantName")
    grantName.text = row['grantName']
    funderCode = ET.SubElement(grant, "funderCode")
    funderCode.text = str(row['funderCode'])
    funderAgency = ET.SubElement(grant, "funderAgency")
    funderAgency.text = row['funderAgency']
    amount = ET.SubElement(grant, "amount")
    amount.text = str(row['amount'])
    startDate = ET.SubElement(grant, "startDate")
    startDate.text = str(row['startDate'])
    endDate = ET.SubElement(grant, "endDate")
    endDate.text = row['endDate']
    keyword = ET.SubElement(grant, "keyword")
    keyword.text = row['keyword']
    description = ET.SubElement(grant, "description")
    description.text = row['description']
    investigator = ET.SubElement(grant, "investigator")
    investigatorId = ET.SubElement(investigator, "investigatorId")
    investigatorId.text = row['investigatorId']

# Create the ElementTree object
tree = ET.ElementTree(root)

# Save the XML to a file
xml_file = 'Grants.xml'
tree.write(xml_file, encoding='utf-8', xml_declaration=True)

print(f"XML saved to '{xml_file}'")

XML saved to 'Grants.xml'
